## Importing dependencies and environmental variables

In [1]:
# Dependencies
import csv as csv
import http.client 
import json
import numpy as np
import os
import pandas as pd
import pprint
import psycopg2
import requests
import sqlalchemy
from bs4 import BeautifulSoup as bs
from dotenv import load_dotenv
from selenium import webdriver
from sodapy import Socrata
from splinter import Browser
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, Float, Date

In [2]:
# Loading homeowrk7.env so that the environment variables can be used in the engine object url
load_dotenv("project3.env")
# Initializing variables to hold each environmet varaible
username=os.environ.get("USERNAME")
password=os.environ.get("PASSWORD")
host=os.environ.get("HOST")
port=os.environ.get("PORT")
database=os.environ.get("DATABASE")
google_key = os.environ.get('GOOGLE_API_KEY')
yelp_client_id = os.environ.get('YELP_CLIENT_ID')
yelp_key = os.environ.get('YELP_API_KEY')
sf_data_key = os.environ.get('SFDATAAPPTOKEN')

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Reading in csv's and concatenating them

In [15]:
sf_coffee_data = pd.read_csv("data/coffee_data/cleaned_sf_coffee_shop_data.csv")
seattle_coffee_data = pd.read_csv("data/coffee_data/cleaned_seattle_coffee_shop_data.csv")
la_coffee_data = pd.read_csv("data/coffee_data/cleaned_los_angeles_coffee_shop_data.csv")

In [16]:
complete_coffee_data = pd.concat([sf_coffee_data,
                                  seattle_coffee_data,
                                  la_coffee_data
                                 ], ignore_index=True, sort=False)

In [17]:
complete_coffee_data = complete_coffee_data.fillna(0)

In [18]:
complete_coffee_data = complete_coffee_data[(complete_coffee_data["city"] != "Emeryville") 
                                            & (complete_coffee_data["city"] != "Sacramento")
                                            & (complete_coffee_data["city"] != "San+rafael")
                                           ]

In [19]:
complete_coffee_data = complete_coffee_data.drop(complete_coffee_data.index[[17, 18, 19, 20]]).reset_index(drop=True)

In [21]:
complete_coffee_data.to_csv("data/coffee_data/complete_coffee_shop_data.csv")

## Compiling lists of zip codes for each city so that demographic data for them can be gathered

In [95]:
sf_zip_codes = complete_coffee_data.loc[complete_coffee_data["city"]=="San Francisco", "zip_code"]
sf_zip_code_list = []
for i in sf_zip_codes:
    if i not in sf_zip_code_list:
        sf_zip_code_list.append(i)
    else:
        continue

In [75]:
seattle_zip_codes = complete_coffee_data.loc[complete_coffee_data["city"]=="Seattle", "zip_code"]
seattle_zip_code_list = []
for i in seattle_zip_codes:
    if i not in seattle_zip_code_list:
        seattle_zip_code_list.append(i)
    else:
        continue

In [84]:
la_zip_codes = complete_coffee_data.loc[complete_coffee_data["city"]=="Los Angeles", "zip_code"]
la_zip_code_list = []
for i in la_zip_codes:
    if i not in la_zip_code_list:
        la_zip_code_list.append(int(i))
    else:
        continue

In [82]:
greater_la_zip_codes = complete_coffee_data.loc[(complete_coffee_data["city"]!="San Francisco") & (complete_coffee_data["city"]!="Seattle"), "zip_code"]
greater_la_zip_code_list = []
for i in greater_la_zip_codes:
    if i not in greater_la_zip_code_list:
        greater_la_zip_code_list.append(int(i))
    else:
        continue